# My version

In [257]:
import numpy as np
import sympy as sp

Remaking Random class

In [258]:
class Random(object):
    __seed = 0
    
    def reset_seed(new_seed):
        Random.__seed = new_seed

    def __init__(self, min_value, max_value, rows, columns):
        self.rand = np.random.default_rng(Random.__seed)
        self.max_value = max_value
        self.min_value = min_value
        self.rows = rows
        self.columns = columns

    def Bool(self):
        return self.rand.integers(2)

    def Sign(self):
        return -1 if self.rand.integers(2) == 0 else 1

    def Bounded(self, allow_zeros):
        result = self.rand.integers(low=self.min_value, high=self.max_value)
        if allow_zeros == True:
            return result
        while result == 0:
            result = self.rand.integers(low=self.min_value, high=self.max_value)
        return result
    
    def RowIndex(self):
        return self.rand.integers(self.rows)
    
    def ColumnIndex(self):
        return self.rand.integers(self.columns)

    def NonEqualIndices(self):
        row = self.rand.integers(self.rows)
        column = self.rand.integers(self.columns)
        while row == column:
            column = self.rand.integers(self.columns)
        return row, column

    def Matrix(self, row, column, allow_zeros):
        result = sp.Matrix(row, column, lambda i, j: self.Bounded(allow_zeros))
        return result


Importing some generators to test

In [259]:
def gen_FirstElementaryM(size, value, row, column):
    result = sp.eye(size)
    result[row, column] = value
    return result

In [260]:
class MatrixForm(object):
    __default_Random = Random(0,0,0,0)
    
    def reset_default_random(min_value, max_value, rows, columns):
        MatrixForm.__default_Random = Random(min_value, max_value, rows, columns)
    
    def __init__(self, **flags):
        self.form = flags["form"]
        self.rand = flags.get("random", MatrixForm.__default_Random)

        if (self.form == "FirstElementary"):
            row = flags.get("row", -1)
            column = flags.get("column", -1)
            if (row == -1):
                row = self.rand.RowIndex()
                while row == column:  
                    row = self.rand.RowIndex()
            if (column == -1):
                column = self.rand.ColumnIndex()
                while row == column:
                    column = self.rand.ColumnIndex()
            if (row == column):
                row, column = self.rand.NonEqualIndices()
            self.data = gen_FirstElementaryM(size=flags["size"], 
                                            value=flags.get("value", self.rand.Bounded(flags.get("allow_zeros", True))),
                                            row=row,
                                            column=column
                                           )

In [261]:
Random.reset_seed(1)

In [262]:
print(Random._Random__seed)

1


In [263]:
R = Random(-5, 5, 3, 3)

In [264]:
M = MatrixForm(form="FirstElementary", allow_zeros=False)

ValueError: low >= high

Default Random causes an error, because low and high both are 0 

So we reset it. In code it will be already good

In [294]:
MatrixForm.reset_default_random(0, 1, 0,1)

In [265]:
M = MatrixForm(form="FirstElementary", size=3, random = R, allow_zeros=False)

In [266]:
M.data

Matrix([
[1, 0, 0],
[0, 1, 4],
[0, 0, 1]])

In [283]:
M = MatrixForm(form="FirstElementary", size=3, random=R, allow_zeros=True)

In [284]:
M.data

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

Now we can reduce the amount of random values

For example, we can decide which row we want

In [285]:
M = MatrixForm(form="FirstElementary", size=3, random=R, allow_zeros=True, row=1)

In [286]:
M.data

Matrix([
[ 1, 0, 0],
[-5, 1, 0],
[ 0, 0, 1]])

In [287]:
M = MatrixForm(form="FirstElementary", size=3, random=R, allow_zeros=False, row=1)

In [288]:
M.data

Matrix([
[1, 0,  0],
[0, 1, -5],
[0, 0,  1]])

Or we can decide which column and value, but make random row

In [289]:
M = MatrixForm(form="FirstElementary", size=3, random=R, allow_zeros=True, column=2, value=6)

In [290]:
M.data

Matrix([
[1, 0, 6],
[0, 1, 0],
[0, 0, 1]])

In [291]:
M = MatrixForm(form="FirstElementary", size=3, random=R, column=2, value=6)

In [292]:
M.data

Matrix([
[1, 0, 0],
[0, 1, 6],
[0, 0, 1]])

As we can see, we do not need to allow zeros, because value is written 

Or we can write everything by hand (so we even do not need to write random=R)

Here we need working default Random, or we will have an error

In [298]:
M = MatrixForm(form="FirstElementary", size=3,  row=1, column=2, value=10)

In [299]:
M.data

Matrix([
[1, 0,  0],
[0, 1, 10],
[0, 0,  1]])